## AI Inference on image dataset retrieved from a cloud
This notebook is an example how to use DeGirum PySDK to do AI inference of an image dataset.
Image dataset is retrieved from the cloud using `fiftyone` API.

This script works with the following inference options:

1. [DeGirum Cloud Platform](https://cs.degirum.com),
1. DeGirum-hosted AI server node shared via Peer-to-Peer VPN,
1. AI server node hosted by you in your local network,
1. AI server running on your local machine,
1. DeGirum ORCA accelerator directly installed on your local machine.

To try different options, you just need to change the `inference_option` in the code below.

### Specify where do you want to run your inferences and dataset parameters here

In [ ]:
inference_option = 1  # <<< change it according to your needs selecting from the list in the header comment

# specify the name of desired dataset to retrieve;
# see https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html
foz_dataset_name = "coco-2017"

# specify list of class labels to retrieve; None for all classes
foz_classes = None # ["car", "cup", "person"]

# specify which splits to download ("train", "validation", "test")
foz_splits = "validation"

# specify # of samples in dataset to retrieve
samples_num = 1000

# specify the model to be used for inference
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca_1"

### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools
import sys, IPython
fo = mytools.import_fiftyone() # import 'fiftyone' package for dataset management

In [ ]:
# download dataset
dataset = fo.zoo.load_zoo_dataset(
    foz_dataset_name,
    dataset_dir="./my-dataset", 
    classes=foz_classes,
    split=foz_splits,
    max_samples=samples_num,
    shuffle=True,
    drop_existing_dataset=False)
print(dataset)

In [ ]:
# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

# load model
model = zoo.load_model(model_name)
model.output_confidence_threshold = 0.1 # set low confidence threshold for proper statistics calculation

In [ ]:
print("Extracting dataset samples...")
all_samples = dataset.head(dataset.count()) # retrieve all dataset samples
all_files = [s.filepath for s in all_samples]
print("...done")

In [ ]:
print("Running inference:")
progress = mytools.Progress(len(all_files))
for n, res in enumerate(model.predict_batch(all_files)):
    if model.image_backend == 'pil':
        w, h = res.image.size
    else: # opencv
        w = res.image.shape[1]
        h = res.image.shape[0]
    
    detections = []
    for box in res.results:
        # Convert to [top-left-x, top-left-y, width, height]
        # in relative coordinates in [0, 1] x [0, 1]
        x1, y1, x2, y2 = box["bbox"]
        rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
        detections.append(fo.Detection(label=box["label"], bounding_box=rel_box, confidence=box["score"]))
    all_samples[n]["predictions"] = fo.Detections(detections=detections)
    all_samples[n].save()
    progress.step()

In [ ]:
# run evaluation on predictions
eval_result = dataset.evaluate_detections("predictions", classes=foz_classes, compute_mAP=True)

In [ ]:
# print some evaluation results
print( f"mAP = {eval_result.mAP():.2f}\n")
eval_result.print_report(classes=foz_classes)